In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models


In [4]:

def conv_block(x, filters, kernel_size, strides, padding='same'):
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding=padding)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def fpn_block(pyramid, filters):
    p5, p4, p3, p2 = pyramid
    p4 = layers.Add()([layers.UpSampling2D()(p5), p4])
    p3 = layers.Add()([layers.UpSampling2D()(p4), p3])
    p2 = layers.Add()([layers.UpSampling2D()(p3), p2])
    return p2, p3, p4, p5

def build_fpn(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    # Backbone network (e.g., ResNet)
    c1 = conv_block(inputs, 64, 3, 1)
    c2 = conv_block(c1, 128, 3, 2)
    c3 = conv_block(c2, 256, 3, 2)
    c4 = conv_block(c3, 512, 3, 2)
    c5 = conv_block(c4, 1024, 3, 2)
    
    # Top-down pathway
    p5 = conv_block(c5, 256, 1, 1)
    p4 = conv_block(c4, 256, 1, 1)
    p3 = conv_block(c3, 256, 1, 1)
    p2 = conv_block(c2, 256, 1, 1)
    
    # Feature Pyramid Network
    p2, p3, p4, p5 = fpn_block([p5, p4, p3, p2], 256)
    
    # Edge detection heads
    edge_p2 = layers.Conv2D(1, 1, activation='sigmoid')(p2)
    edge_p3 = layers.Conv2D(1, 1, activation='sigmoid')(p3)
    edge_p4 = layers.Conv2D(1, 1, activation='sigmoid')(p4)
    edge_p5 = layers.Conv2D(1, 1, activation='sigmoid')(p5)
    
    # Upsample to the same size
    edge_p3 = layers.UpSampling2D()(edge_p3)
    edge_p4 = layers.UpSampling2D(size=(4, 4))(edge_p4)
    edge_p5 = layers.UpSampling2D(size=(8, 8))(edge_p5)
    
    # Concatenate edge maps
    edges = layers.Concatenate()([edge_p2, edge_p3, edge_p4, edge_p5])
    edges = layers.Conv2D(1, 1, activation='sigmoid')(edges)
    
    model = models.Model(inputs, edges)
    return model

# Example usage
input_shape = (256, 256, 3)
model = build_fpn(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
